In [ ]:
import os
import ansys.speos.script as script
from ansys.speos.core import Speos

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

Create connection with speos rpc server

In [ ]:
speos = Speos(host="localhost", port=50098)

Load a speos file into a project feature.

In [ ]:
p = script.Project(speos=speos, path=os.path.join(tests_data_path, "Prism.speos", "Prism.speos"))
print(p)

Preview the project

In [ ]:
p.preview()

Compute

In [ ]:
sim_features = p.find(name="Prism", feature_type=script.Simulation)
sim = sim_features[0]
sim.compute_CPU()

Watch results

In [ ]:
from ansys.speos.workflow.open_result import open_result_image

open_result_image(simulation_feature=sim, result_name="Prism.Irradiance.1.xmp")

Modify irradiance sensor - compute - watch result

Here the irradiance sensor is modified regarding its wavelengths range : from 500 to 600 nm.  
Then the simulation is computed again, and we can see the difference in the output xmp.

In [ ]:
irr_features = p.find(name=".*", name_regex=True, feature_type=script.Sensor.Irradiance)
irr = irr_features[0]
irr.set_irradiance().set_type_spectral().set_wavelengths_range().set_start(500).set_end(600).set_sampling(11)
irr.commit()

sim.compute_CPU()

open_result_image(simulation_feature=sim, result_name="Prism.Irradiance.1.xmp")